In [7]:
import cv2
import json
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [8]:
dataset_dir = './AEC'
category = {
    'train':{
        'img':'.\\AEC_recognition\\train\\img',
        'labels':'.\\AEC_recognition\\train\\labels.json'
    },
    'validation':{
        'img':".\\AEC_recognition\\validation\\img",
        'labels':'.\\AEC_recognition\\validation\\labels.json'
    }
}
minival_dataset = './minival2014'
label_file_names = os.listdir(dataset_dir)
minival_dataset_names = os.listdir(minival_dataset)
image_file_names = [name for name in label_file_names if name.endswith('.jpg')]
trainval_file_name = [name for name in label_file_names if name.endswith('rec')]
minival_file_name = [name for name in minival_dataset_names if name.endswith('jpg')]

In [9]:
def getLabel(label_file_name):
    labels = {}
    for name in label_file_name:
        with open(os.path.join('./AEC', name), encoding='utf-8') as f:
            label = f.read().split('\n')
            labels[name[:-4]] = label
    return labels

trainval_labels = getLabel(trainval_file_name)
print(trainval_labels['aec_0'])

['11*5=55 165 74 345 142 1', '12*3=36 165 160 339 212 1', '13*5=65 163 226 353 272 1', '14*6=84 172 293 357 366 1', '15*7=105 172 369 364 418 1', '16*8=128 172 435 462 486 1', '17*9=153 607 107 791 173 1', '18*6=108 609 183 803 231 1', '19*7=133 613 248 791 301 1', '21*3=63 609 320 781 381 1', '22*5=110 607 390 781 447 1', '23*5=115 609 458 805 525 1', '24*6=144 1063 156 1274 216 1', '25*7=175 1065 234 1261 288 1', '28*6=168 1067 302 1251 354 1', '26*8=188 1065 378 1251 430 1', '27*9=243 1061 456 1267 504 1', '29*7=203 1061 527 1280 584 1', '301*3*3>910 176 700 435 749 1', '17*2*4<160 180 776 419 822 1', '18*9*3<540 184 852 427 896 1', '320*2*4<2400 917 764 1210 853 1', '198*3*7<4200 919 854 1210 927 1', '191*4*5>3800 917 934 1210 1009 1', '4*9=36 354 1263 553 1336 1', '36*2=72 336 1324 582 1408 1', '74*6=444 903 1376 1107 1463 1', '449*2=888 912 1442 1193 1532 1']


In [10]:
def imshow(image):
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [11]:
def save(key,value,file_name,mode,labels):  
    img_save_path = category[mode]['img']  
    img = cv2.imread(file_name)
    # image_id = int(key[key.index('_')+1:])
    for i in range(len(value)):
        element = value[i].split(' ')
        if element[0] == '' or len(element) == 0:
            continue
        else:
            x1,y1,x2,y2 = int(element[1]),int(element[2]),int(element[3]),int(element[4])
            crop_image = img[y1:y2+1,x1:x2+1]
            crop_image_name = key+'_'+str(i)+'.jpg'
            crop_file_path = os.path.join(img_save_path,crop_image_name)
            label = {
                'image_path':crop_image_name,
                'label':element[0]
            }
            try:
                cv2.imwrite(crop_file_path,crop_image)
            except:
                print(crop_image_name)
                continue
            labels.append(label)

In [12]:
def generation(labels):
    train_data = {}
    validation_data = {}
    train_labels = []
    validation_labels = []
    for key,value in tqdm(labels.items(),total=len(labels),ascii=True):
        file_name = os.path.join(dataset_dir,key+'.jpg')
        if key+'.jpg' in minival_file_name:
            save(key,value,file_name,'validation',validation_labels)
        else:
            save(key,value,file_name,'train',train_labels)
    train_data['labels'] = train_labels
    validation_data['labels'] = validation_labels
    train_labels_path = category['train']['labels']
    validation_labels_path = category['validation']['labels']
    with open(train_labels_path,'w') as f:
        json.dump(train_data,f,indent=2)
    with open(validation_labels_path,'w') as f:
        json.dump(validation_data,f,indent=2)
    print('finish')
generation(trainval_labels)

 10%|#         | 514/5000 [00:09<01:35, 47.08it/s]

aec_1457_7.jpg


 78%|#######7  | 3884/5000 [01:13<00:22, 49.10it/s]

aec_4485_27.jpg


 90%|######### | 4520/5000 [01:25<00:06, 69.19it/s]

aec_561_15.jpg


100%|##########| 5000/5000 [01:34<00:00, 53.01it/s]


finish
